In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("/kaggle/input/titanic/train.csv")
test=pd.read_csv("/kaggle/input/titanic/test.csv")
train.sample(3)

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.shape

In [ ]:
sns.countplot(train['Survived'])

In [ ]:
sns.countplot(train['Pclass'])

In [ ]:
sns.countplot(train['Sex'])

In [ ]:
sns.countplot(train['SibSp'])

In [ ]:
sns.countplot(train['Parch'])

In [ ]:
sns.countplot(train['Embarked'])

In [ ]:
sns.distplot(train['Age'])

In [ ]:
sns.distplot(train['Fare'])

In [ ]:
class_fare=train.pivot_table(index='Pclass',values='Fare') #taking mean
class_fare.plot(kind='bar')
plt.xlabel('Pclass')
plt.ylabel('Avg Fare')
plt.xticks(rotation=0)
plt.show()

In [ ]:
class_fare=train.pivot_table(index='Pclass',values='Fare', aggfunc=np.sum)
class_fare.plot(kind='bar')
plt.xlabel('Pclass')
plt.ylabel('Total Fare')
plt.xticks(rotation=0)
plt.show()

In [ ]:
sns.barplot(data=train,x='Pclass',y='Fare',hue='Survived')

In [ ]:
sns.barplot(data=train,x='Survived',y='Fare',hue='Pclass')

## Preprocessing

In [ ]:
#finding null values
train_len=len(train)
df=pd.concat([train,test],axis=0)#combine two dataframe
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df=df.drop(columns=['Cabin'],axis=1)

In [ ]:
df['Age'].mean()

In [ ]:
df['Age']=df['Age'].fillna(df['Age'].mean())
df['Fare']=df['Fare'].fillna(df['Fare'].mean())

In [ ]:
print(df['Embarked'].mode())
print(df['Embarked'].mode()[0])

In [ ]:
df['Embarked']=df['Embarked'].fillna(df['Embarked'].mode()[0])

In [ ]:
sns.distplot(df['Fare'])

In [ ]:
df['Fare']=np.log(df['Fare']+1)

In [ ]:
sns.distplot(df['Fare'])

In [ ]:
corr=df.corr()
plt.figure(figsize=(10,7))
sns.heatmap(corr,annot=True,cmap='coolwarm')

In [ ]:
df.head()

In [ ]:
df=df.drop(columns=['Name','Ticket'],axis=1)

In [ ]:
df.sample(2)

1. ## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols=['Sex','Embarked']
le=LabelEncoder()
for col in cols:
    df[col]=le.fit_transform(df[col])
df.head()

In [ ]:
train=df.iloc[:train_len,:]
test=df.iloc[train_len:,:]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
x=train.drop(columns=['PassengerId','Survived'],axis=1)
y=train['Survived']

In [ ]:
x.head()

## Model Training

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
def training(model,x,y):
    xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42)
    model.fit(xtrain,ytrain)
    print('Accuracy',model.score(xtest,ytest))
    
    score=cross_val_score(model,x,y)
    print('CV score',np.mean(score))

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression()
training(model1,x,y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model2=DecisionTreeClassifier()
training(model2,x,y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model3=RandomForestClassifier()
training(model3,x,y)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model4=ExtraTreesClassifier()
training(model4,x,y)

In [ ]:
from xgboost import XGBClassifier
model5=XGBClassifier()
training(model5,x,y)

In [ ]:
from lightgbm import LGBMClassifier
model6=LGBMClassifier()
training(model6,x,y)

In [ ]:
from catboost import CatBoostClassifier
model7=CatBoostClassifier(verbose=0)
training(model7,x,y)

## Complete model training with Full Data

In [ ]:
model=LGBMClassifier()
model.fit(x,y)

In [ ]:
test.head()

In [ ]:
xtest=test.drop(columns=['PassengerId','Survived'],axis=1)

In [ ]:
xtest.head()

In [ ]:
pred=model.predict(xtest)
pred

## Test Submission

In [ ]:
s=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
s.head()

In [77]:
s.info()

In [76]:
s['Survived']=pred
s['Survived']=s['Survived'].astype('int')

In [78]:
s.info()

In [79]:
s.head()

In [80]:
s.to_csv('submission.csv',index=False)